In [1]:
import xml.etree.ElementTree as ET 
import os
from Preprocessing import Preprocessing
import numpy as np

In [2]:
#Path to input file, before trimming
inputFilename = 'agh-fis-spr17.xml'

#Constraints required, SameAttendees are converted to NotOverlap
hards = ['NotOverlap', 'SameAttendees']
softs = ['DifferentTime', 'DifferentDays']

In [3]:
#Trims the input file and outputs the trimmed filename
outputfile = Preprocessing.getTrimmedFile(inputFilename, hards, softs)

In [4]:
#Extracts the data from trimmed file
courses, subparts, classes, hardConstraints, softConstraints, students = Preprocessing.extractData(outputfile)

1239 Classes extracted
665 Subparts extracted
340 Courses extracted
657 Hard Constraints extracted
3 Soft Constraints extracted
1641 Students extracted


In [5]:
import random

# This loop assigns a random index and the number of available times for each class. This is stored as a tuple in a dictionary using classID as key. This index is always less than the number of timings available with each class.

initial_solution = {}

for classID in classes.keys():
    initial_solution[classID] = (random.randrange(len(classes[classID].availTimes)), len(classes[classID].availTimes))
    #print(solution_template[classID])

In [6]:
#Check for two timings for DifferentTime constraints
def getSameTimeslots(timing1, timing2):
    #getting start and end times for both timings
    t1start = int(timing1.start)
    t2start = int(timing2.start)
    t1end = t1start + int(timing1.length)
    t2end = t2start + int(timing2.length)

    #Total overlap in time regardless of day or week
    overlapLength = 0

    #Calculates the daily overlap if classes occur at any same day
    if(t1end >= t2start):
        overlapLength = t1end - t2start + 1
    elif(t2end >= t1start):
        overlapLength = t2end - t1start + 1

    #print(f"Overlapping timeslots: {overlapLength}")
    #Returns the overlap length
    return overlapLength

In [7]:
#Check for timings for DifferentDay timings 
def getSameDays(timing1, timing2):
    #getting days data from both timings
    t1d = list(timing1.days)
    t2d = list(timing2.days)

    #Count for days overlapped
    count = 0
    #Iterate through all days of the the week
    for i in range(0, 7):
        #Checks if class occur on the same day at any day of the semester
        if (t1d[i] == t2d[i]) and (t1d[i]==1):
            #Add one to days overlapped
            count = count + 1

    #print(f"Overlapping days: {count}")
    #return number of days the classes occur on the same day
    return count

In [8]:
#function to check if 2 timings are overlapping
def isOverlapped(timing1, timing2):
    #getting start and end times for both timings
    t1start = int(timing1.start)
    t2start = int(timing2.start)
    t1end = t1start + int(timing1.length)
    t2end = t2start + int(timing2.length)
    #Count of overlapping days
    count = 0

    
    #number of 5-min timeslots overlapped
    dailyOverlap = 0

    #this if is to check if the time of the classes are overlapping. 
    #classes will overlap only if the time of the class overlap on any day
    if (t1end >= t2start) or (t2end >= t1start):
        #interate through all days of the semester
        for i in range(0, timing1.tdays_np.size):
            #Checks if class occur on the same day at any day of the semester
            if (timing1.tdays_np[i] + timing2.tdays_np[i]) == 2:
                #Add one to days overlapped
                count = count + 1
        #Calculates the daily overlap if classes occur at any same day
        if(count>0):
            if(t1end >= t2start):
                dailyOverlap = t1end - t2start + 1
            elif(t2end >= t1start):
                dailyOverlap = t2end - t1start + 1

    #Total 5-min timeslots overlapped in a semester
    violations = count*dailyOverlap
    #print(f"Total timeslot overlaps: {violations}, daily overlap: {dailyOverlap}, days overlapped: {count}")
    return violations, dailyOverlap, count

In [9]:
from itertools import combinations
#Get Hard penalty
def getHardPenalty(solution):
    #Total violations for all constraints
    totalHardViolations = 0

    #Iterating all hard constraints
    for hard in hardConstraints:
        #Check if type is 'NotOverlap'
        if hard.type == 'NotOverlap':

            #Getting list of classes for the corresponding constraint
            classList = hard.classes
            
            #Getting all possible pairs from the list
            pairs = list(combinations(classList, 2)) 
            
            singleConstraintViolation = 0
            #Iterating through all the pairs
            for pair in pairs:
                #Unpacking thepair
                class1ID, class2ID = pair
                #Extracting class timings
                class1Index,_ = solution[class1ID]
                class2Index,_ = solution[class2ID]
                class1Timing = classes[class1ID].availTimes[class1Index]
                
                class2Timing = classes[class2ID].availTimes[class2Index]

                #Getting 'NonOverlap' violations
                violations, dailyOverlap, count = isOverlapped(class1Timing, class2Timing)
                
                #Adding to total violations
                singleConstraintViolation = singleConstraintViolation + violations
           
            if(singleConstraintViolation>0):
                hard.satisfied = False
            else:
                hard.satisfied = True
           

            totalHardViolations = totalHardViolations + singleConstraintViolation
    #print(f"Total Hard penalty: {totalHardViolations}")
    return totalHardViolations    
            

        

In [10]:
#Get soft penalty
def getSoftPenalty(solution):
    #Total violations for all constraints
    totalSoftPenalty = 0

    #Iterating all hard constraints
    for soft in softConstraints:
        #Getting list of classes for the corresponding constraint
        classList = soft.classes
        penalty = soft.penalty
            
        #Getting all possible pairs from the list
        pairs = list(combinations(classList, 2)) 
            
        constraintTotal = 0
        #Iterating through all the pairs
        for pair in pairs:
            #Unpacking thepair
            class1ID, class2ID = pair
            #Extracting class timings
            class1Index,_ = solution[class1ID]
            class2Index,_ = solution[class2ID]
            class1Timing = classes[class1ID].availTimes[class1Index]
            class2Timing = classes[class2ID].availTimes[class2Index]

            #Check for 'DifferentTime'
            if(soft.type == 'DifferentTime'):
                overlaps = getSameTimeslots(class1Timing, class2Timing)
                constraintTotal = constraintTotal + (overlaps*penalty)
            
            #Check for 'DifferentDays'
            elif(soft.type == 'DifferentDays'):
                count = getSameDays(class1Timing, class2Timing)
                constraintTotal = constraintTotal + (count*penalty)

        if(constraintTotal>0):
            soft.satisfied = False
        else:
            soft.satisfied = True

        #Adding to total soft penalty after multiplying with penalty corresponding to that soft constraint
        totalSoftPenalty = constraintTotal*int(soft.penalty)



    #print(f"Soft Penalty: {totalSoftPenalty}")
    return totalSoftPenalty


In [11]:
#Get penalty of timings
def getTimingPenalty(solution):
    #Timing penalty calculation
    totalTimingPenalty = 0

    #Iterating through all the classes
    for classID in classes.keys():
        #Getting selected timing from each class
        selectedIndex,_ = solution[classID]
        selectedTiming = classes[classID].availTimes[selectedIndex]
        #Adding penalty of selected timing to total timings penalty
        totalTimingPenalty = totalTimingPenalty + int(selectedTiming.penalty)

    #print(f"Timing Penalty: {totalTimingPenalty}")
    return totalTimingPenalty

In [12]:
# Function to mutate a given solution
def mutate(solution):
    #Creating a list of keys(classIDs)
    indexList = list(solution.keys())
    #Getting a random classID from the list
    randomClass = indexList[random.randrange(len(indexList))]

    #Extracting the timing info of that class
    timingIndex,timingsLen = solution[randomClass]

    #Reducing the probabilty of not having a mutation. If a class is selected with a single timing, mutation is not possible. 
    if timingsLen == 1:
        for i in range(10):
            randomClass = indexList[random.randrange(len(indexList))]
            timingIndex,timingsLen = solution[randomClass]
            if(timingsLen>1):
                break
    #Mutating the index
    newIndex = (timingIndex+random.randrange(timingsLen-1)+1)%timingsLen
    print(f"Mutating class {randomClass} from timing index {timingIndex} to {newIndex} out of {timingsLen} timings")

    #Replacing the dictionary value
    solution[randomClass] = (newIndex, timingsLen)
    return solution


In [13]:
#Calculate modified soft penalty
def getModifiedSoftPenalty(solution):
    #Total violations for all constraints
    totalSoftPenalty = 0

    #Iterating all hard constraints
    for soft in softConstraints:
        #Getting list of classes for the corresponding constraint
        classList = soft.classes
        penalty = soft.penalty
            
        #Getting all possible pairs from the list
        pairs = list(combinations(classList, 2)) 
            
        constraintTotal = 0
        #Iterating through all the pairs
        for pair in pairs:
            #Unpacking thepair
            class1ID, class2ID = pair
            #Extracting class timings
            class1Index,_ = solution[class1ID]
            class2Index,_ = solution[class2ID]
            class1Timing = classes[class1ID].availTimes[class1Index]
            class2Timing = classes[class2ID].availTimes[class2Index]

            #Check for 'DifferentTime'
            if(soft.type == 'DifferentTime'):
                overlaps = getSameTimeslots(class1Timing, class2Timing)
                constraintTotal = constraintTotal + (overlaps*penalty)
            
            #Check for 'DifferentDays'
            elif(soft.type == 'DifferentDays'):
                count = getSameDays(class1Timing, class2Timing)
                constraintTotal = constraintTotal + (count*penalty)

        if(constraintTotal>0):
            soft.satisfied = False
        else:
            soft.satisfied = True

        #Adding to total soft penalty after multiplying with penalty corresponding to that soft constraint
        totalSoftPenalty = constraintTotal*int(soft.penalty)*int(soft.scale)



    #print(f"Soft Penalty: {totalSoftPenalty}")
    return totalSoftPenalty


In [14]:
#Calculate modifed hard penalty
def getModifiedHardPenalty(solution):
    #Total violations for all constraints
    totalHardViolations = 0

    #Iterating all hard constraints
    for hard in hardConstraints:
        #Check if type is 'NotOverlap'
        if hard.type == 'NotOverlap':

            #Getting list of classes for the corresponding constraint
            classList = hard.classes
            
            #Getting all possible pairs from the list
            pairs = list(combinations(classList, 2)) 
            
            singleConstraintViolation = 0
            #Iterating through all the pairs
            for pair in pairs:
                #Unpacking thepair
                class1ID, class2ID = pair
                #Extracting class timings
                class1Index,_ = solution[class1ID]
                class2Index,_ = solution[class2ID]
                class1Timing = classes[class1ID].availTimes[class1Index]
                
                class2Timing = classes[class2ID].availTimes[class2Index]

                #Getting 'NonOverlap' violations
                violations, dailyOverlap, count = isOverlapped(class1Timing, class2Timing)
                
                #Adding to total violations
                singleConstraintViolation = singleConstraintViolation + violations
           
            if(singleConstraintViolation>0):
                hard.satisfied = False
            else:
                hard.satisfied = True

            #Adding the 
            totalHardViolations = totalHardViolations + (singleConstraintViolation * hard.scale)
            
    #print(f"Total Hard penalty: {totalHardViolations}")
    return totalHardViolations  

In [15]:
getModifiedHardPenalty(initial_solution)

377259

In [16]:

#Incrementing interations and timeout counts in constraints
def ageConstraints(timeout):
    for hard in hardConstraints:
        if(hard.satisfied):
            hard.iterations = 0
            hard.timeouts = 0
        else:
            hard.iterations = hard.iterations + 1
            if(timeout):
                hard.timeouts = hard.timeouts + 1
    for soft in softConstraints:
        if(soft.satisfied):
            soft.iterations = 0
            soft.timeouts = 0
        else:
            soft.iterations = soft.iterations + 1
            if(timeout):
                soft.timeouts = soft.timeouts + 1

#Scales the constriants if they are unsatisfied
def scaleConstraints(scale):
    for hard in hardConstraints:
        if(hard.satisfied == False):
            hard.scale = hard.scale * scale
    
    for soft in softConstraints:
        if(soft.satisfied == False):
            soft.scale = soft.scale * scale

In [17]:
#Cooling function
def coolTemperature(temperature):
    beta = 1
    return temperature/(1+beta*temperature)

In [18]:
#Return the age of the constraint
def returnAge(hard):
    return hard.timeouts


#Sort the Hardconstraint in descending order of timeouts
def sortHardConstraints():
   hardConstraints.sort(reverse=True,key=returnAge)


#Get the age of the oldest constraint
def getMaxAge():
    sortHardConstraints()
    return hardConstraints[0].timeouts


#Check if the given solution is infeasible
def isInfeasible(solution):
    return (getHardPenalty(solution) > 0)

#Getting the oldest constraints that persisted max timeouts
def getOldestHardConstraints():
    sortHardConstraints()
    return hardConstraints[:3]

In [19]:
import math

#Function to calculate search penalty
def getSearchPenalty(solution):
    return getHardPenalty(solution) + getSoftPenalty(solution) + getTimingPenalty(solution)

#Function to calculate Modifed penalty
def getModifedPenalty(solution):
    return getModifiedHardPenalty(solution) + getModifiedSoftPenalty(solution) + getTimingPenalty(solution)


#Energy function
def calculateEnergy(solution, best):
    gamma = 1
    searchPenalty = getSearchPenalty(solution)
    bestPenalty = getSearchPenalty(best)
    energy = 1 - math.exp(-1*gamma*(searchPenalty - bestPenalty))
    return energy
    
#calculate the acceptance probability of the candidate
def acceptanceProbability(current, candidate, best, temperature):
    currentEnergy = calculateEnergy(current, best)
    candidateEnergy = calculateEnergy(candidate, best)
    if(candidateEnergy < currentEnergy):
        probability = 1
    else:
        probability = math.exp(-1*(candidateEnergy-currentEnergy)/temperature)

    return probability


In [20]:
#Function to get penalty only on a few focused hard constraints
def getFocusedPenalty(solution, focusedConstraints):
    #Total violations for all focused constraints
    totalHardViolations = 0

    #Iterating all hard constraints
    for hard in focusedConstraints:
        #Check if type is 'NotOverlap'
        if hard.type == 'NotOverlap':

            #Getting list of classes for the corresponding constraint
            classList = hard.classes
            
            #Getting all possible pairs from the list
            pairs = list(combinations(classList, 2)) 
            
            singleConstraintViolation = 0
            #Iterating through all the pairs
            for pair in pairs:
                #Unpacking thepair
                class1ID, class2ID = pair
                #Extracting class timings
                class1Index,_ = solution[class1ID]
                class2Index,_ = solution[class2ID]
                class1Timing = classes[class1ID].availTimes[class1Index]
                
                class2Timing = classes[class2ID].availTimes[class2Index]

                #Getting 'NonOverlap' violations
                violations, dailyOverlap, count = isOverlapped(class1Timing, class2Timing)
                
                #Adding to total violations
                singleConstraintViolation = singleConstraintViolation + violations
           
            if(singleConstraintViolation>0):
                hard.satisfied = False
            else:
                hard.satisfied = True
           

            totalHardViolations = totalHardViolations + singleConstraintViolation
    print(f"Focused penalty: {totalHardViolations}")
    return totalHardViolations


In [21]:
import numpy as np

#Perform random walk for a distance

def randomWalk(solution, distance, classSet):

    #Possible steps for any class
    step_set = [-1, 0, 1]

    #perform randomwalk for given distance
    for i in range(distance):
        for eachClass in classSet:
            #Generates a random step with equal probability for each class 
            step = np.random.choice(step_set)
            timingIndex,timingsLen = solution[eachClass]
            #Can have a negative value of just -1. We will then given the index of the last timing.
            if (timingIndex+step) < 0:
                newIndex = timingsLen - 1
            else:
                newIndex = (timingIndex+step)%timingsLen
            
            solution[eachClass] = (newIndex, timingsLen)

    return solution
            



In [22]:
#The function to perform random walk on a focused constraints
def performRandomWalk(solution, focusedConstraints):
    classSet = set()

    #Adding all classes in all the constriants to a set
    for focus in focusedConstraints:
        classList = focus.classes
        for one in classList:
            #Adding to a set since duplicates will be ignored
            classSet.add(one)

    timeout = 0
    timeout_limit = 10

    #Return the solution if a better candidate is not acheived even after timeout_limit iterations
    while(timeout < timeout_limit):
        #Performing random walk for a distance of 5 steps
        candidate = randomWalk(solution, 5, classSet)
        #Check if candidate is better than solution using focused penalty
        if(getFocusedPenalty(candidate, focusedConstraints) < getFocusedPenalty(solution, focusedConstraints)):
            solution = candidate
            timeout = 0
        else:
            timeout = timeout + 1

    return solution



In [23]:
# Start of the SM algorithm

#Extracts the data from trimmed file from start, this will reset all the scales to 1 if scaling was done in any other cell
courses, subparts, classes, hardConstraints, softConstraints, students = Preprocessing.extractData(outputfile)

#Generate a random initial solution
initial_solution = {}

for classID in classes.keys():
    initial_solution[classID] = (random.randrange(len(classes[classID].availTimes)), len(classes[classID].availTimes))

#Computations budget for an algorithm run, this value must be sufficiently high in the order of thousands
budget = 10
#Setting initial temperature
initial_temp = 1000
temp = initial_temp

#Local timeout to 0
local_timeout = 0
timeout_limit = 20
age_limit = 5

#Setting best and current as initial solution
best = initial_solution
current = best

#Setting local best to infinity
local_best = math.inf

#Setting stopping criteria to false
met_criteria = False

i = 0
#Entering SM loop till stopping criteria not met
while(not met_criteria):
    #Cool temperatur
    coolTemperature(temp)
    #Mutate candidate
    candidate = mutate(current)

    #Check if candidate better than best
    if(getSearchPenalty(candidate) < getSearchPenalty(best)):
        best = candidate

    #Check to set new local best
    if(getSearchPenalty(candidate) < local_best):
        local_best = getSearchPenalty(candidate)
        local_timeout = 0
    else:
        local_timeout = local_timeout + 1

    #Checks to accept candidate
    if(getModifedPenalty(candidate) < getModifedPenalty(current)):
        current = candidate
    elif (acceptanceProbability(current, candidate, best, temp) > random.random()):
        current = candidate
    
    #Reset if timeout limit is over
    if(local_timeout > timeout_limit):
        #Reset SM parameters
        temp = initial_temp
        local_best = math.inf
        local_timeout = 0
        #Increment timeouts of unsatisfied constraints
        ageConstraints(True)

        #Get age of the oldest persistent hard constraint
        maxAge = getMaxAge(current)
        
        #If solution is infeasible and the oldest age is more than age_limit, perform random walk
        if(isInfeasible(current) and (maxAge > age_limit)):
            focusedConstraints = getOldestHardConstraints()
            current = performRandomWalk(current, focusedConstraints)
        else:
            #Scale penalties otherwise, this is where MP is getting changed
            scaleConstraints(1.01)

    #Increment to show one iteration
    i = i+1

    #Check for stopping criteria
    if((getSearchPenalty(best)<300) or (i>budget)):
        met_criteria = True

print(best)





1239 Classes extracted
665 Subparts extracted
340 Courses extracted
657 Hard Constraints extracted
3 Soft Constraints extracted
1641 Students extracted
Mutating class 523 from timing index 31 to 21 out of 41 timings
Mutating class 70 from timing index 62 to 29 out of 70 timings
Mutating class 527 from timing index 10 to 57 out of 75 timings
Mutating class 478 from timing index 29 to 13 out of 43 timings
Mutating class 73 from timing index 23 to 9 out of 80 timings
Mutating class 431 from timing index 9 to 17 out of 44 timings
Mutating class 707 from timing index 13 to 50 out of 65 timings
Mutating class 529 from timing index 12 to 32 out of 37 timings
Mutating class 741 from timing index 8 to 1 out of 52 timings
Mutating class 772 from timing index 17 to 3 out of 21 timings
Mutating class 234 from timing index 89 to 162 out of 230 timings
{'1': (61, 230), '2': (44, 151), '3': (56, 245), '4': (205, 245), '5': (230, 245), '6': (134, 245), '7': (101, 245), '8': (28, 181), '9': (47, 215), 

In [42]:
import pandas as pd

lst_str_cols = ['ClassID', 'Start-time', 'End-time', 'Days','Weeks']
# use dictionary comprehension to make dict of dtypes
dict_dtypes = {x : 'str'  for x in lst_str_cols}
df = pd.DataFrame(columns=lst_str_cols)
df.astype(str)
# df.set_index('Attribute',inplace=True)
# df.transpose()
for one in best:
    index,_ = best[one]
    timing = classes[one].availTimes[index]
    end = timing.start + int(timing.length)
    row = [one, timing.start, end, timing.days, timing.week]
    df.loc[len(df)] = row



In [43]:
df.to_csv("solution.csv", index=False)

PermissionError: [Errno 13] Permission denied: 'solution.csv'

In [35]:
for one in best:
    print(one)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
